# BipartiteData example

In [1]:
# Add pytwoway to system path, do not run this
# import sys
# sys.path.append('../../..')

In [2]:
# Import the pytwoway package 
# (Make sure you have installed it using pip install pytwoway)
import pytwoway as tw

## Simulate some data

The package contains functions to simulate data. We use this here to keep things simple.

In [3]:
# For the example, we simulate data
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
display(sim_data)

,wid,fid,comp,year
0,0,75,-0.309731,1
1,0,14,-1.207119,2
2,0,14,-2.163811,3
3,0,22,-1.553568,4
4,0,84,0.086559,5
...,...,...,...,...
49995,9999,171,0.302781,1
49996,9999,181,1.820684,2
49997,9999,181,1.685579,3
49998,9999,181,1.533217,4


## BipartiteData

BipartiteData is a superclass with 3 subclasses:
- BipartiteLong
- BipartiteLongCollapsed
- BipartiteEventStudy

The user interfaces with BipartiteData which converts between the 3 subclasses.

In [4]:
# Our data is in long form (each row gives a single observation)
bd = tw.BipartiteData(sim_data, formatting='long') # Long is default formatting, most data is long
bd.clean_data() # Should always clean data after initializing

In [5]:
# Let's see how our data looks
display(bd.data)

,wid,fid,comp,year
0,0,75,-0.309731,1
1,0,14,-1.207119,2
2,0,14,-2.163811,3
3,0,22,-1.553568,4
4,0,84,0.086559,5
...,...,...,...,...
49995,9999,171,0.302781,1
49996,9999,181,1.820684,2
49997,9999,181,1.685579,3
49998,9999,181,1.533217,4


## Converting formats

In [6]:
# While our original data is long, we might want it to be in event study form (each row gives two consecutive observations)
bd.long_to_es()
display(bd.data)

,wid,f1i,f2i,y1,y2,year_1,year_2,m
0,21,39,39,-1.151285,-1.151285,1,1,0
1,21,39,39,1.654527,1.654527,2,2,0
2,21,39,39,-0.294581,-0.294581,3,3,0
3,21,39,39,-0.468204,-0.468204,4,4,0
4,21,39,39,0.418363,0.418363,5,5,0
...,...,...,...,...,...,...,...,...
40646,9998,93,85,0.941243,0.449756,4,5,1
40647,9999,171,181,0.302781,1.820684,1,2,1
40648,9999,181,181,1.820684,1.685579,2,3,1
40649,9999,181,181,1.685579,1.533217,3,4,1


In [7]:
# We can also use event study data to retrive cross section data (cs=1 gives y1 as y1 for both stayers and movers; cs=0 gives y2 as y1 for only movers - this allows (almost) all income data to be accessed from the y1 column. Note that for movers, the last observation for each worker is not available without manipulation as it is shifted to the y2 column. Also note that the y1 row contains duplicates for all mover income, except for the first period.)
display(bd.get_cs())

,wid,f1i,f2i,y1,y2,year_1,year_2,m,cs
0,21,39,39,-1.151285,-1.151285,1,1,0,1
1,21,39,39,1.654527,1.654527,2,2,0,1
2,21,39,39,-0.294581,-0.294581,3,3,0,1
3,21,39,39,-0.468204,-0.468204,4,4,0,1
4,21,39,39,0.418363,0.418363,5,5,0,1
...,...,...,...,...,...,...,...,...,...
78042,9998,85,93,0.449756,0.941243,5,4,1,0
78043,9999,181,171,1.820684,0.302781,2,1,1,0
78044,9999,181,181,1.685579,1.820684,3,2,1,0
78045,9999,181,181,1.533217,1.685579,4,3,1,0


In [8]:
# Maybe we want to convert back into long form
bd.es_to_long()
display(bd.data)

,wid,fid,comp,year,m
0,0,75,-0.309731,1,1
1,0,14,-1.207119,2,1
2,0,14,-2.163811,3,1
3,0,22,-1.553568,4,1
4,0,84,0.086559,5,1
...,...,...,...,...,...
49995,9999,171,0.302781,1,1
49996,9999,181,1.820684,2,1
49997,9999,181,1.685579,3,1
49998,9999,181,1.533217,4,1


In [9]:
# Now suppose we want to collapse by employment spells (so any consecutive observations with the same worker in the same firm are collapsed into 1 observation)
bd.long_to_collapsed_long()
display(bd.data)

,wid,fid,comp,year_start,year_end,weight,m
0,0,75,-0.309731,1,1,1,1
1,0,14,-1.685465,2,3,2,1
2,0,22,-1.553568,4,4,1,1
3,0,84,0.086559,5,5,1,1
4,1,178,2.587676,1,1,1,1
...,...,...,...,...,...,...,...
29732,9998,45,-1.700584,3,3,1,1
29733,9998,93,0.941243,4,4,1,1
29734,9998,85,0.449756,5,5,1,1
29735,9999,171,0.302781,1,1,1,1


In [10]:
# We can then check out the event study using collapsed data
bd.collapsed_long_to_es()
display(bd.data)

,wid,f1i,f2i,y1,y2,year_start_1,year_end_1,year_start_2,year_end_2,w1,w2,m
0,21,39,39,0.031764,0.031764,1,5,1,5,5,5,0
1,24,102,102,0.181221,0.181221,1,5,1,5,5,5,0
2,38,29,29,-0.999334,-0.999334,1,5,1,5,5,5,0
3,49,52,52,0.186486,0.186486,1,5,1,5,5,5,0
4,60,116,116,1.307871,1.307871,1,5,1,5,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20383,9997,33,48,-0.192398,-0.928684,4,4,5,5,1,1,1
20384,9998,90,45,-0.030269,-1.700584,1,2,3,3,2,1,1
20385,9998,45,93,-1.700584,0.941243,3,3,4,4,1,1,1
20386,9998,93,85,0.941243,0.449756,4,4,5,5,1,1,1


In [11]:
# We can then go back to collapsed long
bd.es_to_collapsed_long()
display(bd.data)

,wid,fid,comp,year_start,year_end,weight,m
0,0,75,-0.309731,1,1,1,1
1,0,14,-1.685465,2,3,2,1
2,0,22,-1.553568,4,4,1,1
3,0,84,0.086559,5,5,1,1
4,1,178,2.587676,1,1,1,1
...,...,...,...,...,...,...,...
29732,9998,45,-1.700584,3,3,1,1
29733,9998,93,0.941243,4,4,1,1
29734,9998,85,0.449756,5,5,1,1
29735,9999,171,0.302781,1,1,1,1


## Clustering

In [12]:
# Starting over
bd = tw.BipartiteData(sim_data, formatting='long')
bd.clean_data()
display(bd.data)

,wid,fid,comp,year
0,0,75,-0.309731,1
1,0,14,-1.207119,2
2,0,14,-2.163811,3
3,0,22,-1.553568,4
4,0,84,0.086559,5
...,...,...,...,...
49995,9999,171,0.302781,1
49996,9999,181,1.820684,2
49997,9999,181,1.685579,3
49998,9999,181,1.533217,4


In [13]:
# We can cluster from any format and clusters stay when reformatting
bd.cluster()
display(bd.data)

,wid,fid,comp,year,j,m
0,0,75,-0.309731,1,9,1
1,0,14,-1.207119,2,0,1
2,0,14,-2.163811,3,0,1
3,0,22,-1.553568,4,0,1
4,0,84,0.086559,5,8,1
...,...,...,...,...,...,...
49995,9999,171,0.302781,1,2,1
49996,9999,181,1.820684,2,2,1
49997,9999,181,1.685579,3,2,1
49998,9999,181,1.533217,4,2,1


In [14]:
bd.long_to_es()
display(bd.data)

,wid,f1i,f2i,y1,y2,year_1,year_2,j1,j2,m
0,21,39,39,-1.151285,-1.151285,1,1,6,6,0
1,21,39,39,1.654527,1.654527,2,2,6,6,0
2,21,39,39,-0.294581,-0.294581,3,3,6,6,0
3,21,39,39,-0.468204,-0.468204,4,4,6,6,0
4,21,39,39,0.418363,0.418363,5,5,6,6,0
...,...,...,...,...,...,...,...,...,...,...
40646,9998,93,85,0.941243,0.449756,4,5,4,4,1
40647,9999,171,181,0.302781,1.820684,1,2,2,2,1
40648,9999,181,181,1.820684,1.685579,2,3,2,2,1
40649,9999,181,181,1.685579,1.533217,3,4,2,2,1


In [15]:
bd.es_to_long()
bd.long_to_collapsed_long()
display(bd.data)

,wid,fid,comp,year_start,year_end,weight,j,m
0,0,75,-0.309731,1,1,1,9,1
1,0,14,-1.685465,2,3,2,0,1
2,0,22,-1.553568,4,4,1,0,1
3,0,84,0.086559,5,5,1,8,1
4,1,178,2.587676,1,1,1,2,1
...,...,...,...,...,...,...,...,...
29732,9998,45,-1.700584,3,3,1,3,1
29733,9998,93,0.941243,4,4,1,4,1
29734,9998,85,0.449756,5,5,1,4,1
29735,9999,171,0.302781,1,1,1,2,1


In [16]:
bd.collapsed_long_to_es()
display(bd.data)

,wid,f1i,f2i,y1,y2,year_start_1,year_end_1,year_start_2,year_end_2,w1,w2,j1,j2,m
0,21,39,39,0.031764,0.031764,1,5,1,5,5,5,6,6,0
1,24,102,102,0.181221,0.181221,1,5,1,5,5,5,8,8,0
2,38,29,29,-0.999334,-0.999334,1,5,1,5,5,5,6,6,0
3,49,52,52,0.186486,0.186486,1,5,1,5,5,5,3,3,0
4,60,116,116,1.307871,1.307871,1,5,1,5,5,5,8,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20383,9997,33,48,-0.192398,-0.928684,4,4,5,5,1,1,6,3,1
20384,9998,90,45,-0.030269,-1.700584,1,2,3,3,2,1,4,3,1
20385,9998,45,93,-1.700584,0.941243,3,3,4,4,1,1,3,4,1
20386,9998,93,85,0.941243,0.449756,4,4,5,5,1,1,4,4,1


## Clustering options

In [17]:
# We can cluster on a specific year in long form
bd = tw.BipartiteData(sim_data, formatting='long')
bd.clean_data()
bd.cluster({'year': 1})
display(bd.data)

,wid,fid,comp,year,j,m
0,0,75,-0.309731,1,9,1
1,0,14,-1.207119,2,2,1
2,0,14,-2.163811,3,2,1
3,0,22,-1.553568,4,2,1
4,0,84,0.086559,5,0,1
...,...,...,...,...,...,...
49995,9999,171,0.302781,1,1,1
49996,9999,181,1.820684,2,1,1
49997,9999,181,1.685579,3,1,1
49998,9999,181,1.533217,4,1,1


In [18]:
# We can cluster on movers or stayers in any form
# Note that not all firms are clustered, so some rows have NaNs
bd = tw.BipartiteData(sim_data, formatting='long')
bd.clean_data()
bd.cluster({'stayers_movers': 'stayers'})
display(bd.data)

,wid,fid,comp,year,j,m
0,0,75,-0.309731,1,0,1
1,0,14,-1.207119,2,7,1
2,0,14,-2.163811,3,7,1
3,0,22,-1.553568,4,6,1
4,0,84,0.086559,5,3,1
...,...,...,...,...,...,...
49995,9999,171,0.302781,1,1,1
49996,9999,181,1.820684,2,1,1
49997,9999,181,1.685579,3,1,1
49998,9999,181,1.533217,4,1,1


In [19]:
# We can cluster on movers or stayers in any form
# Set 'dropna': True if you want to drop firms that don't get clustered (note that this fully resets firm and worker ids, since they must be contiguous for the FE/CRE/BLM estimators to work)
bd = tw.BipartiteData(sim_data, formatting='long')
bd.clean_data()
bd.cluster({'stayers_movers': 'stayers', 'dropna': True})
display(bd.data)

,wid,fid,comp,year,j,m
0,0,75,-0.309731,1,2,1
1,0,14,-1.207119,2,7,1
2,0,14,-2.163811,3,7,1
3,0,22,-1.553568,4,1,1
4,0,84,0.086559,5,8,1
...,...,...,...,...,...,...
47957,9960,171,0.302781,1,3,1
47958,9960,181,1.820684,2,3,1
47959,9960,181,1.685579,3,3,1
47960,9960,181,1.533217,4,3,1


In [20]:
# We can cluster on movers or stayers in any form
bd = tw.BipartiteData(sim_data, formatting='long')
bd.clean_data()
bd.cluster({'stayers_movers': 'movers'})
display(bd.data)

,wid,fid,comp,year,j,m
0,0,75,-0.309731,1,7,1
1,0,14,-1.207119,2,8,1
2,0,14,-2.163811,3,8,1
3,0,22,-1.553568,4,8,1
4,0,84,0.086559,5,9,1
...,...,...,...,...,...,...
49995,9999,171,0.302781,1,1,1
49996,9999,181,1.820684,2,1,1
49997,9999,181,1.685579,3,1,1
49998,9999,181,1.533217,4,1,1


## Some extra features

In [21]:
# We can get some summary statistics by printing the BipartiteData object
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data = sim_data.rename({'fid': 'f1i'}, axis=1)
bd = tw.BipartiteData(sim_data, col_dict={'fid': 'f1i'}, formatting='long')
bd.clean_data()
display(bd)

format: long
number of workers: 10000
number of firms: 195
number of observations: 50000
mean wage: 0.3862867070374669
max wage: 5.7617660677298925
min wage: -5.285373932814096
collapsed by spell: False
connected: True
contiguous firm ids: True
contiguous worker ids: True
contiguous cluster ids (None if not clustered): None
no nans: True
no duplicates: True

In [22]:
# BipartiteData does some nice column inference
# Suppose some of our columns are mislabeled - create a col_dict to clarify which columns to correct
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data = sim_data.rename({'fid': 'f1i'}, axis=1)
bd = tw.BipartiteData(sim_data, col_dict={'fid': 'f1i'}, formatting='long')
bd.clean_data()
display(bd.data)

,wid,fid,comp,year
0,0,87,1.422830,1
1,0,87,1.661140,2
2,0,87,1.267665,3
3,0,87,2.550360,4
4,0,118,0.599082,5
...,...,...,...,...
49995,9999,21,-0.672321,1
49996,9999,21,-1.377369,2
49997,9999,21,-0.970314,3
49998,9999,21,-0.593032,4


In [23]:
# Suppose we have already clustered - the class determines this automatically
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data['j'] = 1
bd = tw.BipartiteData(sim_data, formatting='long')
bd.clean_data()
display(bd.data)

,wid,fid,comp,year,j
0,0,162,2.896741,1,1
1,0,63,2.465399,2,1
2,0,63,1.037162,3,1
3,0,193,1.818901,4,1
4,0,127,0.645893,5,1
...,...,...,...,...,...
49995,9999,163,1.556137,1,1
49996,9999,163,0.798808,2,1
49997,9999,163,1.662931,3,1
49998,9999,149,1.522641,4,1


In [24]:
# Now suppose we mislabeled our clusters - the class drops this column
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data['j1'] = 1
bd = tw.BipartiteData(sim_data, formatting='long')
bd.clean_data()
display(bd.data)

,wid,fid,comp,year
0,0,173,1.123302,1
1,0,173,1.362065,2
2,0,173,2.285556,3
3,0,64,0.097504,4
4,0,28,-1.346189,5
...,...,...,...,...
49995,9999,151,2.957512,1
49996,9999,84,0.708481,2
49997,9999,152,1.265314,3
49998,9999,63,1.052698,4


In [25]:
# We can make sure our columns are included by specifying the corrected labels in col_dict (note that if the variable of interest requires 2 columns, e.g. weight 1 and weight 2 for event study data, BOTH columns are required - if only one column is included, it is automatically dropped.)
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data['j1'] = 1
bd = tw.BipartiteData(sim_data, col_dict={'j': 'j1'}, formatting='long')
bd.clean_data()
display(bd.data)

,wid,fid,comp,year,j
0,0,7,-1.670336,1,1
1,0,135,0.047628,2,1
2,0,135,-0.065207,3,1
3,0,119,0.985969,4,1
4,0,119,1.923032,5,1
...,...,...,...,...,...
49995,9999,168,3.703281,1,1
49996,9999,139,1.040093,2,1
49997,9999,139,1.721457,3,1
49998,9999,139,-0.024310,4,1


In [26]:
# We may also specify a col_dict that overwrites a column label that would normally be inferred, and inference is skipped
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data = sim_data.rename({'fid': 'j'}, axis=1)
bd = tw.BipartiteData(sim_data, col_dict={'fid': 'j'}, formatting='long')
# Note that even though we have a column named j, because we specify it is the 'fid' column it is not inferred as the cluster values
bd.clean_data()
display(bd.data)

,wid,fid,comp,year
0,0,101,-0.495414,1
1,0,29,-0.445265,2
2,0,29,-0.513969,3
3,0,0,-1.435105,4
4,0,37,0.530158,5
...,...,...,...,...
49995,9999,57,-0.751823,1
49996,9999,57,-1.241723,2
49997,9999,57,1.111050,3
49998,9999,25,-1.414595,4
